<a href="https://colab.research.google.com/github/pthread/code/blob/main/01_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference under FHE for the MNIST Dataset using helayers

In this demo we show how to run a Neural Network under encryption. The Neural Network is loaded as a tensor circuit: a circuit where each node is an tensor operator (matrix multiplication, sum-over-dim, convultion, etc...), and the inputs and outputs of each node are tensors.

Under encryption we convert each tensor to a tile tensor: a data structure where the tensor is broken up into parts or tile, each tile stored encrypted in a ciphertext. Tile tensors support the same set of operators as tensors, and by configuring their shapes we can choose different packing options. This allows the system to easily convert the tensor circuit into a tile tensor circuit, and then an optimization step chooses the best shapes for them. Usually some auxiliary nodes need to be added as well: bootstrapping, duplication, cleanup of junk values from unused slots, etc.

In this demo, we deal with a classification problem for the MNIST dataset [1], trying to correctly classify a batch of samples using a neural network model that will be created and trained using the Keras library (with architecture similar to reference [2]).

We first build a plain neural network for the MNIST model that is based on Cryptonets [2]. Then, we'll encrypt the trained network and run inference over it using FHE. Larger model architetures are available in [HELayers](https://ibm.github.io/helayers/) samples. Specifically this sample is based on `09_Neural_network_MNIST` therein.

In [ ]:
import numpy as np

# Import timers
from timeit import default_timer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf

# Import Keras
from tensorflow.keras import backend as K
from tensorflow.keras import utils, losses
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation
from tensorflow.keras.models import Sequential

# Import the MNIST dataset
from tensorflow.keras.datasets import mnist

import os
import sys

import h5py

train_batch_size = 500
epochs = 10

#### Step 1. Preparing the data - loading and preprocessing the MNIST Dataset.

In this tutorial we use the MNIST dataset that uses samples of dimension $28 \times 28 \times 1$, which we pad with zeros to a dimension of $29 \times 29 \times 1$.

In [ ]:
def process_data(l):
    l = np.expand_dims(l.astype('float32'), -1)
    l /= 255
    return np.pad(l, ((0, 0), (0, 1), (0, 1), (0, 0)))

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = process_data(x_train)
x_test = process_data(x_test)

We generate a validation set

In [ ]:
testSize=16
x_val = x_test[:-testSize]
x_test = x_test[-testSize:]
y_val = y_test[:-testSize]
y_test = y_test[-testSize:]

and we process the labels to have a one-hot representation.

In [ ]:
num_classes = 10
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)
y_val = utils.to_categorical(y_val, num_classes)

Finally, we save the data as h5 files in the `output_dir` directory.

In [ ]:
output_dir = 'out'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def save_data_set(x, y, data_type, s=''):
    fname=os.path.join(output_dir, f'x_{data_type}{s}.h5')
    print("Saving x_{} of shape {} in {}".format(data_type, x.shape,fname))
    xf = h5py.File(fname, 'w')
    xf.create_dataset('x_{}'.format(data_type), data=x)
    xf.close()

    yf = h5py.File(os.path.join(output_dir, f'y_{data_type}{s}.h5'), 'w')
    yf.create_dataset(f'y_{data_type}', data=y)
    yf.close()

save_data_set(x_test, y_test, data_type='test')
save_data_set(x_train, y_train, data_type='train')
save_data_set(x_val, y_val, data_type='val')

Saving x_test of shape (16, 29, 29, 1) in out/x_test.h5
Saving x_train of shape (60000, 29, 29, 1) in out/x_train.h5
Saving x_val of shape (9984, 29, 29, 1) in out/x_val.h5


### Step 2. Building the NN model

For the demonstartion, we use a simple [cryptonets model](https://proceedings.mlr.press/v48/gilad-bachrach16.html) that was the first HE-friendly NN. I.e., a NN where all the operations are polynomials. The network includes one convolutional layer with a kernel of size $5 \times 5$, and subsequently two fully connected (FC) layers separated by square activations.

In [ ]:
class SquareActivation(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.math.square(inputs)

model = Sequential()
model.add(Conv2D(filters=5, kernel_size=5, strides=2, padding='valid', input_shape=x_train[0].shape))
model.add(Flatten())
model.add(SquareActivation())
model.add(Dense(100))
model.add(SquareActivation())
model.add(Dense(num_classes))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 13, 13, 5)         130       
                                                                 
 flatten_2 (Flatten)         (None, 845)               0         
                                                                 
 square_activation_4 (Squar  (None, 845)               0         
 eActivation)                                                    
                                                                 
 dense_4 (Dense)             (None, 100)               84600     
                                                                 
 square_activation_5 (Squar  (None, 100)               0         
 eActivation)                                                    
                                                                 
 dense_5 (Dense)             (None, 10)               

#### Training the network

For training the network we use the Adam optimizer where we set the loss function to be the sum of the square errors.

In [ ]:
def sum_squared_error(y_true, y_pred):
    return K.sum(K.square(y_pred - y_true), axis=-1)

model.compile(loss=sum_squared_error, optimizer='Adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=train_batch_size, epochs=epochs, verbose=2, validation_data=(x_val, y_val), shuffle=True)

# Compute and print the test score
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: { score[0]:.3f}')
print(f'Test accuracy: {score[1] * 100:.3f}%')

Epoch 1/10
120/120 - 8s - loss: 0.3830 - accuracy: 0.8482 - val_loss: 0.2028 - val_accuracy: 0.9508 - 8s/epoch - 65ms/step
Epoch 2/10
120/120 - 5s - loss: 0.1749 - accuracy: 0.9598 - val_loss: 0.1467 - val_accuracy: 0.9686 - 5s/epoch - 42ms/step
Epoch 3/10
120/120 - 5s - loss: 0.1366 - accuracy: 0.9706 - val_loss: 0.1237 - val_accuracy: 0.9727 - 5s/epoch - 41ms/step
Epoch 4/10
120/120 - 7s - loss: 0.1177 - accuracy: 0.9757 - val_loss: 0.1116 - val_accuracy: 0.9761 - 7s/epoch - 58ms/step
Epoch 5/10
120/120 - 5s - loss: 0.1058 - accuracy: 0.9786 - val_loss: 0.1037 - val_accuracy: 0.9774 - 5s/epoch - 40ms/step
Epoch 6/10
120/120 - 7s - loss: 0.0975 - accuracy: 0.9811 - val_loss: 0.0962 - val_accuracy: 0.9787 - 7s/epoch - 55ms/step
Epoch 7/10
120/120 - 5s - loss: 0.0914 - accuracy: 0.9826 - val_loss: 0.0918 - val_accuracy: 0.9808 - 5s/epoch - 44ms/step
Epoch 8/10
120/120 - 5s - loss: 0.0868 - accuracy: 0.9841 - val_loss: 0.0891 - val_accuracy: 0.9813 - 5s/epoch - 40ms/step
Epoch 9/10
120/1

#### Serializing the model (arch and weights)

In [ ]:
# Serialize the model architecture
model_json = model.to_json()
with open(os.path.join(output_dir, 'model.json'), "w") as json_file:
    json_file.write(model_json)

# Serialize the model weights to HDF5
model.save_weights(os.path.join(output_dir, 'model.h5'))

# Step 3 Running the model using FHE

Here we start by loading a plain NN model (nnp) load its content from the `json` and `h5` files.
The `hyper_params` object can be used for various flags, e.g., we can tell the system to just load the architecture and use random weights. Here we keep it empty.

In [ ]:
!pip install pyhelayers
import pyhelayers
print('Imported pyhelayers',pyhelayers.VERSION)

hyper_params = pyhelayers.PlainModelHyperParams()

#initialize the NN architecture and weights from the json and h5 files that we stored before.
nnp = pyhelayers.NeuralNetPlain()
nnp.init_from_files(hyper_params, [os.path.join(output_dir, "model.json"), os.path.join(output_dir, "model.h5")])

Imported pyhelayers 1.5.3.0


We are now ready to build an encrypted version of our NN model using tile tensors. However, as we saw defining the right shapes can be a hard task. Specifically, when the configuration includes multiple HE-related parameters, many of them depend on one another and on the specific NN model we defined.  

To avoid this complexity, we will use the HE profile optimizer. This optimizer finds a configuration that is guaranteed to be secure and feasible. Furthermore, the optimizer receives the plain NN we have built and considers what's optimal for this very model in terms of performance.

We can notify the optimizer of various requirements we have for running the model, with respect to the library and packaging considerations. For example here we ask to optimize for a batch size of 16 samples.

The optimizer is called when compiling the model, given HE run requirements and the plain model. The result returned is a model `profile`, describing how the library and the encrypted NN should be initialized.

In [ ]:
pred_batch_size=16

he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.DefaultContext()])
he_run_req.optimize_for_batch_size(pred_batch_size)

# Set the requirements and run the model
profile = pyhelayers.HeModel.compile(nnp, he_run_req)
print(profile)

HE profileHe configuration requirement:
Security level: 128
Integer part precision: 10
Fractional part precision: 50
Number of slots: 8192
Multiplication depth: 6
Bootstrappable: False
Automatic bootstrapping: False
Rotation keys policy: custom, 15 keys required:
[-512, -256, -128, -64, -32, -16, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
HE context name: SEAL_CKKS
Mode: predict
Tile layout: ( 8 x 64 x 16 )
Mode name: conv_image_to_col
Is model encrypted: true
Using circuit optimization: false
Lazy encoding: false
Handle overflow: false
Base chain index: 6
Use AES inputs: false
Use generically-packed inputs: false
Optimal batch size: 16
Model measuresRequired bootstrap operations: 0
Estimated predict CPU time (s): 3.27
Estimated init model CPU time (s): 4.15
Estimated encrypt input CPU time (s): 1.02
Estimated decrypt output CPU time (s): 0.00
Estimated throughput (samples/s): 4.89
Estimated model memory (MB): 384.07
Estimated input memory (MB): 102.77
Estimated output memory (MB): 0

The HE profile includes a set of requirements from the library, such as the security level, precision, size of ciphertext, multiplication depth etc. Some of these can be set by the user and some others were found by the optimizer based on the NN architecture we defined. We'll now take this requirement object and use it to initialize the library.

In [ ]:
context = pyhelayers.HeModel.create_context(profile)

We are ready to initialize the HE NN and populate its weights. Since we have already built a plain NN describing the model architecture and containing the weights, we'll simply encrypt into an encrypted version of the NN.

Notice we provide the profile that was recommended by the optimizer.

In [ ]:
nn = pyhelayers.NeuralNet(context)
nn.encode_encrypt(nnp, profile)

### Step 4 running an inference operation
We now load and classify real samples of the MNIST dataset from the stored HDF5 files, where we extract the first batch of samples and labels.

In [ ]:
with h5py.File(os.path.join(output_dir, "x_test.h5")) as f:
    x_test = np.array(f["x_test"])
with h5py.File(os.path.join(output_dir, "y_test.h5")) as f:
    y_test = np.array(f["y_test"])

In [ ]:
def extract_batch(x_test, y_test, batch_size, batch_num):
    num_samples = x_test.shape[0]
    num_lebels = y_test.shape[0]

     # assert same size
    assert(num_samples == num_lebels)

    # calc start and end index
    start_index = batch_num * batch_size
    if start_index >= num_samples:
        raise RuntimeError('Not enough samples for batch number ' +
                           str(batch_num) + ' when batch size is ' + str(batch_size))
    end_index = min(start_index + batch_size, num_samples)

    batch_x = x_test.take(indices=range(start_index, end_index), axis=0)
    batch_y = y_test.take(indices=range(start_index, end_index), axis=0)

    return (batch_x, batch_y)

plain_samples, labels = extract_batch(x_test, y_test, pred_batch_size, 0)
print('Batch of size',pred_batch_size,'loaded')

Batch of size 16 loaded


To populate the input, we need to encode and then encrypt the values of the plain input under HE.

In [ ]:
iop = nn.create_io_processor()
samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(samples,[plain_samples])
print('Test data encrypted')

Test data encrypted


We now go ahead with the inference itself. We run the encrypted input through the encrypted NN to obtain encrypted results. This computation does not use the secret key and acts on completely encrypted values. Running the inference is done using the "predict" method of the NN, that receives the encrypted input tile tensor and produces as encrypted output tile tensor.

In [ ]:
startTime = default_timer()

predictions = pyhelayers.EncryptedData(context)
nn.predict(predictions,samples)

latency    = round(default_timer() - startTime,5)
amortized_latency = round(latency/pred_batch_size,5)
print(f"Latency = {latency} seconds, Amortized latency={amortized_latency} seconds")

Latency = 7.89263 seconds, Amortized latency=0.49329 seconds


In order to assess the results of the computation, we first need to decrypt them. This is done by an IO processor that has the secret key and is capable of decrypting the ciphertext and decoding it into plaintext version of the HE computation result.

In [ ]:
plain_predictions = iop.decrypt_decode_output(predictions)

Let's verify the results are close to what we'd get from the plain network:

In [ ]:
expected_pred=nnp.predict([plain_samples])
diff=expected_pred-plain_predictions
print('L2 distance between HE and plain predictions',np.linalg.norm(diff))

L2 distance between HE and plain predictions 2.417642015376116e-09


Let's repeat the optimization and inference, but now for a different batch size:

In [ ]:
pred_batch_size=64


he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.DefaultContext()])
he_run_req.optimize_for_batch_size(pred_batch_size)

# Set the requirements and run the model
profile = pyhelayers.HeModel.compile(nnp, he_run_req)
print(profile)

context = pyhelayers.HeModel.create_context(profile)

nn = pyhelayers.NeuralNet(context)
nn.encode_encrypt(nnp, profile)

plain_samples, labels = extract_batch(x_test, y_test, pred_batch_size, 0)
print('Batch of size',pred_batch_size,'loaded')

iop = nn.create_io_processor()
samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(samples,[plain_samples])

startTime = default_timer()

predictions = pyhelayers.EncryptedData(context)
nn.predict(predictions,samples)

latency    = round(default_timer() - startTime,5)
amortized_latency = round(latency/pred_batch_size,5)
print(f"Latency = {latency} seconds, Amortized latency={amortized_latency} seconds")

HE profileHe configuration requirement:
Security level: 128
Integer part precision: 10
Fractional part precision: 50
Number of slots: 8192
Multiplication depth: 6
Bootstrappable: False
Automatic bootstrapping: False
Rotation keys policy: custom, 12 keys required:
[-1024, -512, -256, -128, -64, 64, 128, 256, 512, 1024, 2048, 4096]
HE context name: SEAL_CKKS
Mode: predict
Tile layout: ( 4 x 32 x 64 )
Mode name: conv_image_to_col
Is model encrypted: true
Using circuit optimization: false
Lazy encoding: false
Handle overflow: false
Base chain index: 6
Use AES inputs: false
Use generically-packed inputs: false
Optimal batch size: 64
Model measuresRequired bootstrap operations: 0
Estimated predict CPU time (s): 6.26
Estimated init model CPU time (s): 14.14
Estimated encrypt input CPU time (s): 3.43
Estimated decrypt output CPU time (s): 0.00
Estimated throughput (samples/s): 10.22
Estimated model memory (MB): 1313.72
Estimated input memory (MB): 346.84
Estimated output memory (MB): 0.26
Esti

<br>

References:

<sub><sup> 1.	LeCun, Yann and Cortes, Corinna. "MNIST handwritten digit database." (2010): </sup></sub>

<sub><sup> 2.	Gilad-Bachrach, R., Dowlin, N., Laine, K., Lauter, K., Naehrig, M. &amp; Wernsing, J.. (2016). CryptoNets: Applying Neural Networks to Encrypted Data with High Throughput and Accuracy. Proceedings of The 33rd International Conference on Machine Learning, in Proceedings of Machine Learning Research 48:201-210 Available from https://proceedings.mlr.press/v48/gilad-bachrach16.html.
</sup></sub>